### We are going to train neural networks on properly labelled data as well as data randomly labelled! From what I understand about neural network, if network is appropriately big enough, network would learn a mapping for that data too! However, given the lack of structure in how the labels are assigned, we want to poke upon if there is different in convergence timings for both datasets!

In [1]:
%matplotlib inline

import os
import numpy as np
from PIL import Image  
from keras import models,layers
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
RAW_PATH = '/Users/gursharan/Desktop/cat-and-dog/'

In [3]:
CAT_PATH = os.path.join(RAW_PATH,'cats')
DOG_PATH = os.path.join(RAW_PATH,'dogs')

In [4]:
cat_images = []
dog_images = []

total_per_category_image = 400
input_size = (256,256)
total_examples = total_per_category_image * 2
total_train = int(total_examples*0.6)
total_valid = int(total_examples*0.2)
total_test  = int(total_examples*0.2)

count = 0
for index,imagepath in enumerate(os.listdir(CAT_PATH)):
    if not 'jpg' in imagepath:
        continue
    cat_img = Image.open(os.path.join(CAT_PATH, imagepath))
    cat_img = cat_img.resize(input_size)
    cat_images.append( np.array( cat_img ) )
    if count == total_per_category_image:
        break

count = 0
for index,imagepath in enumerate(os.listdir(DOG_PATH)):
    if not 'jpg' in imagepath:
        continue
    dog_img = Image.open(os.path.join(DOG_PATH, imagepath))
    dog_img = dog_img.resize(input_size)
    dog_images.append( np.array( dog_img ) )
    count = count + 1
    if count == total_per_category_image:
        break

In [10]:
def model():
    model = models.Sequential()
    model.add( layers.Conv2D(128, 5, input_shape=(256,256,3)))
    model.add( layers.Conv2D(64, 3))
    model.add( layers.Flatten())
    model.add( layers.Dense(32))
    model.add( layers.Dense(1, activation='sigmoid') )
    model.compile( optimizer='rmsprop', loss='binary_crossentropy' )
    return model

In [11]:
def get_correct_data():
    X = []
    Y = []
    for cat in cat_images:
        X.append(cat)
        Y.append(0) #0 for cat
    for dog in dog_images:
        X.append(dog)
        Y.append(1) #1 for dog
    permutation = np.random.permutation(2*total_per_category_image)
    X = np.array(X)[permutation]
    Y = np.array(Y)[permutation]
    
    X_train, X_valid, X_test = X[:total_train], X[total_train:total_train+total_valid], X[total_train+total_valid:]
    Y_train, Y_valid, Y_test = Y[:total_train], Y[total_train:total_train+total_valid], Y[total_train+total_valid:]
    
    return( (X_train, Y_train), (X_valid,Y_valid), (X_test,Y_test) )

def get_random_data():
    X = []
    Y = []
    for cat in cat_images:
        X.append(cat)
        Y.append(np.random.randint(2))
    for dog in dog_images:
        X.append(dog)
        Y.append(np.random.randint(2))
    
    permutation = np.random.permutation(2*total_per_category_image)
    X = np.array(X)[permutation]
    Y = np.array(Y)[permutation]
    
    X_train, X_valid, X_test = X[:total_train], X[total_train:total_train+total_valid], X[total_train+total_valid:]
    Y_train, Y_valid, Y_test = Y[:total_train], Y[total_train:total_train+total_valid], Y[total_train+total_valid:]
        
    return( (X_train, Y_train), (X_valid,Y_valid), (X_test,Y_test) )

In [ ]:
(X_train, Y_train), (X_valid,Y_valid), (X_test,Y_test) = get_correct_data()
model_correct = model()
history_correct = model_correct.fit(X_train, Y_train, batch_size=32, epochs=10, 
                  validation_data=(X_valid,Y_valid))
evaluation = model_correct.evaluate(X_test,Y_test)
print('Evaluation on test data is :', evaluation)

In [ ]:
(X_train, Y_train), (X_valid,Y_valid), (X_test,Y_test) = get_random_data()
model_random = model()
history_random = model_random.fit(X_train, Y_train, batch_size=32, epochs=20, 
                  validation_data=(X_valid,Y_valid))
evaluation_random = model_random.evaluate(X_test,Y_test)
print('Evaluation on test data is :', evaluation_random)